In [5]:
import ee

In [22]:
dataset = "emodnet"


In [23]:
ee.Initialize()

In [28]:
geom =  ee.Geometry.Polygon([
    [
            [-174.52128906250002, 79.52559147277664],
            [-174.52128906250002, 2.61899508391546],
            [164.38496093749998, 2.61899508391546],
            [164.38496093749998, 79.52559147277664]
        ]
    ], 
    None, 
    False
)
print(geom.getInfo())
land = ee.Image("users/gena/land_polygons_image")
bathys = {
    "gebco": ee.Image("users/gena/GEBCO_2014_2D"),
    "emodnet": ee.Image("users/gena/EMODNET")
}

bathy = bathys[dataset]
batch = 3
bucket_folder = dataset

{'geodesic': False, 'type': 'Polygon', 'coordinates': [[[-174.52128906250002, 2.61899508391546], [164.38496093749998, 2.61899508391546], [164.38496093749998, 79.52559147277664], [-174.52128906250002, 79.52559147277664], [-174.52128906250002, 2.61899508391546]]]}


In [29]:
# Units in m
# Upsample (bathy is rather coarse, so we make scale a bit)
upsample = 10

resolution = 10

tileSize = 256 * resolution * upsample
radius = tileSize / 2

# focal max -> buffer with 5px  -> 250000 / 50 ->  500m -> 2500m buffer.
# use 500m resolution for buffer
landDilated = land.mask().focal_max(5).reproject(**{ "crs": 'EPSG:4326', "scale": radius / 5 })

# lookup region where bathy is undeep  and not above SL and not in buffered land polygon
mask = (
    bathy
    .gt(-50)
    .And(bathy.lt(-1))
    .And(landDilated.Not())
)

# Create a large number of samples
# Filter samples by region
# Like computing pi by throwing darts at a dartboard in a square.... 
# but then less efficient.
samples = (
    bathy
    .updateMask(mask)
    # divide by the factor 
    .divide(upsample)
    .sample(**{ 
        "region": geom.getInfo(), 
        'scale': 1000, 
        "numPixels": 50000, 
        "seed": 42 + batch, 
        "dropNulls": True, 
        "geometries": True
    })
)

print(samples.size().getInfo())

# Draw boxes around the buffer (consider sampling by sqrt(2) and  then sampling rotated)
def draw_box(f):
    return f.buffer(radius).bounds()
export_rects = samples.map(draw_box)


# Export tile to bucket
def export_dem(i, bounds):
    task = ee.batch.Export.image.toCloudStorage(**{
        "image": bathy, 
        "description": f'bathymetry-{i + batch * 10000}',
        "bucket": 'bathy_sample',
        "fileNamePrefix": f'{bucket_folder}/bathymetry-{i + batch * 10000}', 
        "dimensions": '256x256', 
        "region": bounds, 
    })
    return task

export_rects_list = export_rects.toList(export_rects.size())

def export_dem_feature(i):
    task = export_dem(i, ee.Feature(export_rects_list.get(i)).geometry())
    return task

tasks = []
for f in ee.List.sequence(0, export_rects.size()).getInfo():
    task = export_dem_feature(f)
    tasks.append(task)

112


In [30]:
for task in tasks:
    task.start()

In [27]:
for task in tasks:
    print(task.status())

{'state': 'READY', 'description': 'bathymetry-20000', 'creation_timestamp_ms': 1625570405611, 'update_timestamp_ms': 1625570405611, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_IMAGE', 'id': 'Y2YER3QAKAB5WCH6XG62NT6Z', 'name': 'projects/earthengine-legacy/operations/Y2YER3QAKAB5WCH6XG62NT6Z'}
{'state': 'READY', 'description': 'bathymetry-20001', 'creation_timestamp_ms': 1625570406446, 'update_timestamp_ms': 1625570406446, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_IMAGE', 'id': 'IBJW4VW3ZADTK7OQEOLTCTR2', 'name': 'projects/earthengine-legacy/operations/IBJW4VW3ZADTK7OQEOLTCTR2'}
{'state': 'READY', 'description': 'bathymetry-20002', 'creation_timestamp_ms': 1625570406907, 'update_timestamp_ms': 1625570406907, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_IMAGE', 'id': 'KS2YF2TSDNSHIA5NK5RNLQ6U', 'name': 'projects/earthengine-legacy/operations/KS2YF2TSDNSHIA5NK5RNLQ6U'}
{'state': 'READY', 'description': 'bathymetry-20003', 'creation_timestamp_ms': 1625570407333, 'update_timestamp_ms'

{'state': 'READY', 'description': 'bathymetry-20028', 'creation_timestamp_ms': 1625570420436, 'update_timestamp_ms': 1625570420436, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_IMAGE', 'id': 'LFIBHZECYQ2US7BWAFIG7S6L', 'name': 'projects/earthengine-legacy/operations/LFIBHZECYQ2US7BWAFIG7S6L'}
{'state': 'READY', 'description': 'bathymetry-20029', 'creation_timestamp_ms': 1625570420861, 'update_timestamp_ms': 1625570420861, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_IMAGE', 'id': 'RBQPAK3GBZFMPPDC4LACUZ4H', 'name': 'projects/earthengine-legacy/operations/RBQPAK3GBZFMPPDC4LACUZ4H'}
{'state': 'READY', 'description': 'bathymetry-20030', 'creation_timestamp_ms': 1625570421306, 'update_timestamp_ms': 1625570421306, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_IMAGE', 'id': 'CAPCDQKWHMH237RQYN4BUUFE', 'name': 'projects/earthengine-legacy/operations/CAPCDQKWHMH237RQYN4BUUFE'}
{'state': 'READY', 'description': 'bathymetry-20031', 'creation_timestamp_ms': 1625570421741, 'update_timestamp_ms'

{'state': 'READY', 'description': 'bathymetry-20057', 'creation_timestamp_ms': 1625570434275, 'update_timestamp_ms': 1625570434275, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_IMAGE', 'id': 'KTWUMOB7KR62FZDI5QLWWDKT', 'name': 'projects/earthengine-legacy/operations/KTWUMOB7KR62FZDI5QLWWDKT'}
{'state': 'READY', 'description': 'bathymetry-20058', 'creation_timestamp_ms': 1625570434734, 'update_timestamp_ms': 1625570434734, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_IMAGE', 'id': 'SIIQI64D4J6PU3APTM3DRDIS', 'name': 'projects/earthengine-legacy/operations/SIIQI64D4J6PU3APTM3DRDIS'}
{'state': 'READY', 'description': 'bathymetry-20059', 'creation_timestamp_ms': 1625570435214, 'update_timestamp_ms': 1625570435214, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_IMAGE', 'id': 'RNJL355VUUSOSIFOWYYSD7XG', 'name': 'projects/earthengine-legacy/operations/RNJL355VUUSOSIFOWYYSD7XG'}
{'state': 'READY', 'description': 'bathymetry-20060', 'creation_timestamp_ms': 1625570436572, 'update_timestamp_ms'

{'state': 'READY', 'description': 'bathymetry-20085', 'creation_timestamp_ms': 1625570451650, 'update_timestamp_ms': 1625570451650, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_IMAGE', 'id': 'HPF7BYNGQM344BYGR5NSHHND', 'name': 'projects/earthengine-legacy/operations/HPF7BYNGQM344BYGR5NSHHND'}
{'state': 'READY', 'description': 'bathymetry-20086', 'creation_timestamp_ms': 1625570452122, 'update_timestamp_ms': 1625570452122, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_IMAGE', 'id': 'JZMA25RU2JLAFU6I4VN2ARD7', 'name': 'projects/earthengine-legacy/operations/JZMA25RU2JLAFU6I4VN2ARD7'}
{'state': 'READY', 'description': 'bathymetry-20087', 'creation_timestamp_ms': 1625570452552, 'update_timestamp_ms': 1625570452552, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_IMAGE', 'id': 'E7AVD7PNK4FAR2A42DW4BMTB', 'name': 'projects/earthengine-legacy/operations/E7AVD7PNK4FAR2A42DW4BMTB'}
{'state': 'READY', 'description': 'bathymetry-20088', 'creation_timestamp_ms': 1625570453016, 'update_timestamp_ms'

{'state': 'READY', 'description': 'bathymetry-20113', 'creation_timestamp_ms': 1625570467775, 'update_timestamp_ms': 1625570467775, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_IMAGE', 'id': 'RGTVQBK2OYXQ6EVJE2RLVZQA', 'name': 'projects/earthengine-legacy/operations/RGTVQBK2OYXQ6EVJE2RLVZQA'}
{'state': 'READY', 'description': 'bathymetry-20114', 'creation_timestamp_ms': 1625570468196, 'update_timestamp_ms': 1625570468196, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_IMAGE', 'id': '65ZKCTNMOE4I5Z2WPRFX2GNM', 'name': 'projects/earthengine-legacy/operations/65ZKCTNMOE4I5Z2WPRFX2GNM'}
